<a href="https://colab.research.google.com/github/khuramgill/Face-Embaddings/blob/main/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install pinecone

  Using cached pinecone_plugin_inference-2.0.1-py3-none-any.whl.metadata (2.2 kB)
Using cached pinecone_plugin_inference-2.0.1-py3-none-any.whl (84 kB)
  Attempting uninstall: pinecone-plugin-inference
    Found existing installation: pinecone-plugin-inference 1.1.0
    Uninstalling pinecone-plugin-inference-1.1.0:
      Successfully uninstalled pinecone-plugin-inference-1.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pinecone-client 5.0.1 requires pinecone-plugin-inference<2.0.0,>=1.0.3, but you have pinecone-plugin-inference 2.0.1 which is incompatible.


In [22]:
from pinecone import Pinecone

pc = Pinecone(api_key="pcsk_2agnRW_PJGcuYNduPh4LXPttu1bnT31UninNBGbyZqKDUiCDAxdY7asgDJkLHmR3KCDJzP")
index = pc.Index("new")

# **Now testing Image Vectors**

In [28]:
!pip install open_clip_torch pinecone-client


  Using cached pinecone_plugin_inference-1.1.0-py3-none-any.whl.metadata (2.2 kB)
Using cached pinecone_plugin_inference-1.1.0-py3-none-any.whl (85 kB)
  Attempting uninstall: pinecone-plugin-inference
    Found existing installation: pinecone-plugin-inference 2.0.1
    Uninstalling pinecone-plugin-inference-2.0.1:
      Successfully uninstalled pinecone-plugin-inference-2.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pinecone 5.4.0 requires pinecone-plugin-inference<3.0.0,>=2.0.0, but you have pinecone-plugin-inference 1.1.0 which is incompatible.


In [29]:
!pip install --upgrade pinecone-client


In [37]:
!pip list | grep -E 'Pillow|torch|pinecone|open_clip'


open_clip_torch                    2.29.0
pinecone                           5.4.0
pinecone-client                    5.0.1
pinecone-plugin-inference          1.1.0
pinecone-plugin-interface          0.0.7
torch                              2.5.1+cu121
torchaudio                         2.5.1+cu121
torchsummary                       1.5.1
torchvision                        0.20.1+cu121


In [2]:
from PIL import Image
import torch
import pinecone
import open_clip


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)


Using device: cpu


In [5]:
from pinecone import Pinecone

pc = Pinecone(api_key="pcsk_2agnRW_PJGcuYNduPh4LXPttu1bnT31UninNBGbyZqKDUiCDAxdY7asgDJkLHmR3KCDJzP")
index = pc.Index("new")

# Connect


load Model and its fun

In [6]:
model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms(
    'hf-hub:laion/CLIP-ViT-B-32-laion2B-s34B-b79K'
)
tokenizer = open_clip.get_tokenizer('hf-hub:laion/CLIP-ViT-B-32-laion2B-s34B-b79K')

model = model.to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


open_clip_pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

open_clip_config.json:   0%|          | 0.00/604 [00:00<?, ?B/s]

In [7]:
image_path = '/content/2.jpg'
image = Image.open(image_path).convert('RGB')
preprocessed_image = preprocess_val(image).unsqueeze(0).to(device)


Generate Embaddings


In [8]:
with torch.no_grad():
    image_embedding = model.encode_image(preprocessed_image).cpu().numpy().flatten()


In [9]:
index.upsert(
    vectors=[
        {
            "id": "image1",
            "values": image_embedding.tolist(),
            "metadata": {"type": "image", "description": "Sample image"}
        }
    ],
    namespace="image_namespace"
)


{'upserted_count': 1}

Upload Multiple Images

In [13]:
import requests
from PIL import Image
import io

# List of public image URLs
image_urls = [
    "https://drive.google.com/uc?id=1nIrw9H-NDXhjO_8K0_Myzjp8tT_fONxW",
    "https://drive.google.com/uc?id=1XMjTZ4NtKMkHXgJwhbBlpmbGL0QROTDA",
    "https://drive.google.com/uc?id=1J6ALJHD2MSc5tI5Sod7r3AdlUnCsWQxu",
    "https://drive.google.com/uc?id=1peTtrAZIFAX3g46erMA6aZoeSSXzVwrO",
    "https://drive.google.com/uc?id=1uZe1BWLCuBLC5kQ2yq_VC5UhtpEm1vuN",
    "https://drive.google.com/uc?id=1J4Cq5o8OTPr3lacf3GG4vX4XuYsBK1ns",
    "https://drive.google.com/uc?id=1UN_LOwKydi_2dhlvv4nR00iTjr3PEGsu",
    "https://drive.google.com/uc?id=1TSkrlof8XXjwhwFyj_WiSMIfXqf6QLiZ",
    "https://drive.google.com/uc?id=1psngmmlrVROetpkaZ1DU4p5wTDOvo5q0",
    "https://drive.google.com/uc?id=1XKFViNQOk2LCvYI5Qp7blVcnlRMn97y9",
    "https://drive.google.com/uc?id=1Vt8IFHhKe8m3jo0ymj7WL1QA0nPuO9da",
    "https://drive.google.com/uc?id=1eP5PSO2_0jiK_xK1X4d2jimLrryj4jGE",
    "https://drive.google.com/uc?id=1tAUhhywMeFYM9GBV6XqYyfbAokVsz6S7",
    "https://drive.google.com/uc?id=1aLH-Lwe8liEb6BivWogcs8BaF526e-g1",
    "https://drive.google.com/uc?id=1XYTzMI6eqZDo_DmSoXuwhwseL884btes",
    "https://drive.google.com/uc?id=1jB4FT-esHARBI0wFkgM2pqpPEb2x_eJr",
    "https://drive.google.com/uc?id=11P0kZQr3L4CSzRd5Rlrj7CoWeLM4J92z",
    "https://drive.google.com/uc?id=1KaQNi10CJEpYAfAR3z-HceLy3AXJLs6g",
    "https://drive.google.com/uc?id=1hSmMW3jsGUIrGBZ9rSDc3i65qrdAww-C",
    "https://drive.google.com/uc?id=1gf0XwXmxAhadQGt9bJ2LHeuzhOtBxqaJ",
    "https://drive.google.com/uc?id=1WQJielsQ0cugpCF0srrkf7clTsxO8-Qo",
    "https://drive.google.com/uc?id=1SD00dQGxuuy7iEw-6Kv5YI0gPFrkg-Cr",
    "https://drive.google.com/uc?id=1DErnGjOtIQkRsXbdihyiHSa8K7iQtZmp",
    "https://drive.google.com/uc?id=108KUJ-v2Tpajosf15AT752NCpP6e1bPS",
    "https://drive.google.com/uc?id=1E9TcUXwgMafWUBM2u0bE-e_ivgMASBlX",
    "https://drive.google.com/uc?id=1DgdcfC4OrHljMD1Rl9pW5bWXZ2RBN7hz",
    "https://drive.google.com/uc?id=1fK1tddE6qXyXRx4jMq51SVoBKdSWAKdf",
    "https://drive.google.com/uc?id=1rBJblGgAQNtoPYtVR6dYvOOFw1uwM_7o",
    "https://drive.google.com/uc?id=1lnGQAsjqa-8NPWp3c-TO7fV76tNp3txO",
    "https://drive.google.com/uc?id=1uirBTH35SxbA30H8N3zgK64WrmJIWSBh",
    "https://drive.google.com/uc?id=1bmY2oQqpy8KDQCEd6Q68D7VBVWgq8e8-",
    "https://drive.google.com/uc?id=1y6WfYI5vhoCZU2KYp7bkWTuJnmX_LoHs",
    "https://drive.google.com/uc?id=1PiXH1DUVtuJmhsnKUA5dE7ggCiMi06RT",
    "https://drive.google.com/uc?id=1ujG8n-8pOvuN2ZHo1KdVY4p3h0deIm4w",
    "https://drive.google.com/uc?id=1lcDFPn3snYIX_67NrXkLTUP8YzJT_iPz",
    "https://drive.google.com/uc?id=1DJbXe5Mwp6U3w5WTupWrTT8YEyRFzUPs",
    "https://drive.google.com/uc?id=1EZIHmMNGAWaYkmRVIJyQGyP1Y_2oL-lZ",
    "https://drive.google.com/uc?id=1Hf1KV0QmsMvymk5kpWqWVQMxj5wJSU-c",
    "https://drive.google.com/uc?id=19vo3_OyxWp0kLvqeoS_A1uW7RDXGTgPn",
    "https://drive.google.com/uc?id=1K18NSwnCiruE-fdsuRd1AH6XX7vlzCJc",
    "https://drive.google.com/uc?id=1cWog2eKXO5pFURqJPcxBe6JERF3hdixj",
    "https://drive.google.com/uc?id=1b9ZK6eh_aKLt_KthDLoUU-RQnW-e73pp",
    "https://drive.google.com/uc?id=1-g2NvmmvMVhMfwFDIeIByfst9_XSpDie",
    "https://drive.google.com/uc?id=1gJyXgMloiC2tbwU7m-VQ63C7MGx3vhO3",
    "https://drive.google.com/uc?id=1CDmXwCjokpnQrjkDvZoZILP5e-7SnMae",
    "https://drive.google.com/uc?id=1txBPZV2YaMy9hOzceJmNss8rni8NecYZ",
    "https://drive.google.com/uc?id=1YXh9SoA9cS0Oea7HRD1uoEV4HCIw2fan",
    "https://drive.google.com/uc?id=167rTHr5U_zhQofmzYv0PwTn-jQyiezNT",
    "https://drive.google.com/uc?id=1IVhXcsCbMFMZPqdhYovwN_UqXi3MrO77",
    "https://drive.google.com/uc?id=1c7V4eAFBmULEKf6Gbz1JsIiPiSZRN3F2"
]

vectors = []

for i, url in enumerate(image_urls):
    try:
        response = requests.get(url)
        response.raise_for_status()

        image = Image.open(io.BytesIO(response.content)).convert('RGB')
        preprocessed_image = preprocess_val(image).unsqueeze(0).to(device)

        with torch.no_grad():
            image_embedding = model.encode_image(preprocessed_image).cpu().numpy().flatten()

        vectors.append({
            "id": f"image_{i}",
            "values": image_embedding.tolist(),
            "metadata": {"type": "image", "url": url, "description": f"Image {i}"}
        })
    except Exception as e:
        print(f"Error processing image {url}: {e}")

index.upsert(vectors=vectors, namespace="image_namespace")


{'upserted_count': 50}

In [15]:
import requests
from PIL import Image
import torch

# Function to download image from URL
def download_image(url):
    response = requests.get(url, stream=True)
    response.raise_for_status()
    return Image.open(response.raw).convert("RGB")


user_image_url = input("Enter the image URL for querying: ")

query_image = download_image(user_image_url)
query_image = preprocess_val(query_image).unsqueeze(0).to(device)

with torch.no_grad():
    query_embedding = model.encode_image(query_image).cpu().numpy().flatten()

response = index.query(
    vector=query_embedding.tolist(),
    top_k=5,
    include_metadata=True,
    namespace="image_namespace"
)

# Display results
print("Query Results:")
for match in response["matches"]:
    print(f"ID: {match['id']}, Score: {match['score']}, Metadata: {match['metadata']}")


Enter the image URL for querying: https://drive.google.com/uc?id=1K18NSwnCiruE-fdsuRd1AH6XX7vlzCJc
Query Results:
ID: image_39, Score: 0.99999994, Metadata: {'description': 'Image 39', 'type': 'image', 'url': 'https://drive.google.com/uc?id=1K18NSwnCiruE-fdsuRd1AH6XX7vlzCJc'}
ID: image_10, Score: 0.862163365, Metadata: {'description': 'Image 10', 'type': 'image', 'url': 'https://drive.google.com/uc?id=1Vt8IFHhKe8m3jo0ymj7WL1QA0nPuO9da'}
ID: image_12, Score: 0.839465559, Metadata: {'description': 'Image 12', 'type': 'image', 'url': 'https://drive.google.com/uc?id=1tAUhhywMeFYM9GBV6XqYyfbAokVsz6S7'}
ID: image_45, Score: 0.826702654, Metadata: {'description': 'Image 45', 'type': 'image', 'url': 'https://drive.google.com/uc?id=1txBPZV2YaMy9hOzceJmNss8rni8NecYZ'}
ID: image_42, Score: 0.825977445, Metadata: {'description': 'Image 42', 'type': 'image', 'url': 'https://drive.google.com/uc?id=1-g2NvmmvMVhMfwFDIeIByfst9_XSpDie'}
